## Cambiar el tipo de datos de las columnas en Pandas.

Tienes tres opciones principales para convertir tipos en pandas:

* **to_numeric()** - proporciona funcionalidad para convertir de forma segura tipos no numéricos (por ejemplo, cadenas) a un tipo numérico adecuado. (Ver también to_datetime() y to_timedelta() .)

* **astype()** - convierte (casi) cualquier tipo a (casi) cualquier otro tipo (incluso si no es necesariamente sensato hacerlo). También le permite convertir a categorial tipos (muy útil).

* **infer_objects()** - un método de utilidad para convertir columnas de objetos que contienen objetos de Python a un tipo de pandas si es posible.

https://www.it-swarm.dev/es/python/cambiar-el-tipo-de-datos-de-las-columnas-en-pandas./1072113742/

### 1. to_numeric()
La mejor manera de convertir una o más columnas de un DataFrame a valores numéricos es usar pandas.to_numeric() .

Esta función intentará cambiar los objetos no numéricos (como cadenas) en números enteros o de punto flotante, según corresponda.

#### Uso básico
La entrada a `to_numeric()` es una serie o una sola columna de un DataFrame.

In [6]:
import pandas as pd
a = [['a', '1.2', '4.2'], ['b', '70', '0.03'], ['x', '5', '0']]
df = pd.DataFrame(a)

In [7]:

s = pd.Series(["8", 6, "7.5", 3, "0.9"]) # mixed string and numeric values
s

0      8
1      6
2    7.5
3      3
4    0.9
dtype: object

In [8]:
 pd.to_numeric(s) # convert everything to float values

0    8.0
1    6.0
2    7.5
3    3.0
4    0.9
dtype: float64

Como puede ver, se devuelve una nueva serie. Recuerde asignar esta salida a una variable o nombre de columna para continuar usándola:

In [ ]:
# convert Series
my_series = pd.to_numeric(my_series)

# convert column "a" of a DataFrame
df["a"] = pd.to_numeric(df["a"])

# También puede usarlo para convertir varias columnas de un DataFrame a través del método apply():

# convert all columns of DataFrame
df = df.apply(pd.to_numeric) # convert all columns of DataFrame

# convert just columns "a" and "b"
df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)

Mientras sus valores puedan ser convertidos, eso es probablemente todo lo que necesita.



In [ ]:
Manejo de errores

Pero, ¿qué pasa si algunos valores no se pueden convertir a un tipo numérico?

to_numeric() también toma un argumento de palabra clave errors que le permite forzar que los valores no numéricos sean NaN, o simplemente ignorar las columnas que contienen estos valores.

Aquí hay un ejemplo que usa una serie de cadenas s que tiene el tipo de objeto:

>>> s = pd.Series(['1', '2', '4.7', 'pandas', '10'])
>>> s
0         1
1         2
2       4.7
3    pandas
4        10
dtype: object
El comportamiento predeterminado es aumentar si no puede convertir un valor. En este caso, no puede hacer frente a la cadena 'pandas':

>>> pd.to_numeric(s) # or pd.to_numeric(s, errors='raise')
ValueError: Unable to parse string
En lugar de fallar, podríamos querer que 'pandas' se considere un valor numérico faltante/malo. Podemos forzar valores no válidos a NaN de la siguiente manera usando el argumento de la palabra clave errors:

>>> pd.to_numeric(s, errors='coerce')
0     1.0
1     2.0
2     4.7
3     NaN
4    10.0
dtype: float64
La tercera opción para errors es ignorar la operación si se encuentra un valor no válido:

>>> pd.to_numeric(s, errors='ignore')
# the original Series is returned untouched
Esta última opción es particularmente útil cuando desea convertir todo su DataFrame, pero no sabe cuál de nuestras columnas se puede convertir de manera confiable a un tipo numérico. En ese caso solo escribe:

df.apply(pd.to_numeric, errors='ignore')
La función se aplicará a cada columna del DataFrame. Las columnas que se pueden convertir a un tipo numérico se convertirán, mientras que las columnas que no pueden (por ejemplo, que contienen cadenas o fechas sin dígitos) se dejarán solas.

Downcasting
De forma predeterminada, la conversión con to_numeric() le dará un dtype int64 o float64 (o cualquier ancho de entero que sea nativo de su plataforma).

Eso suele ser lo que quiere, pero ¿qué sucede si desea guardar algo de memoria y usar un tipo de letra más compacto, como float32, o int8?

to_numeric() le da la opción de cambiar a 'entero', 'firmado', 'sin firmar', 'flotar'. Aquí hay un ejemplo para una serie simple s de tipo entero:

>>> s = pd.Series([1, 2, -7])
>>> s
0    1
1    2
2   -7
dtype: int64
La reducción a "entero" utiliza el entero más pequeño posible que puede contener los valores:

>>> pd.to_numeric(s, downcast='integer')
0    1
1    2
2   -7
dtype: int8
La reducción a 'flotar' elige de manera similar un tipo flotante más pequeño que el normal:

>>> pd.to_numeric(s, downcast='float')
0    1.0
1    2.0
2   -7.0
dtype: float32
2. astype()
El método astype() le permite ser explícito sobre el tipo de dato que desea que tenga su DataFrame o Series. Es muy versátil, ya que puede intentar ir de un tipo a otro.

Uso básico
Simplemente elija un tipo: puede usar un tipo de tipo NumPy (por ejemplo, np.int16), algunos tipos de Python (por ejemplo, bool) o tipos específicos de pandas (como el tipo de categórico).

Llame al método en el objeto que desea convertir y astype() intentará convertirlo por usted:

# convert all DataFrame columns to the int64 dtype
df = df.astype(int)

# convert column "a" to int64 dtype and "b" to complex type
df = df.astype({"a": int, "b": complex})

# convert Series to float16 type
s = s.astype(np.float16)

# convert Series to Python strings
s = s.astype(str)

# convert Series to categorical type - see docs for more details
s = s.astype('category')
Observe que dije "intentar": si astype() no sabe cómo convertir un valor en la serie o el marco de datos, generará un error. Por ejemplo, si tiene un valor NaN o inf, obtendrá un error al intentar convertirlo en un entero.

A partir de las pandas 0.20.0, este error se puede eliminar al pasar errors='ignore'. Su objeto original será devuelto sin tocar.

Ten cuidado
astype() es potente, pero a veces convertirá los valores "incorrectamente". Por ejemplo:

>>> s = pd.Series([1, 2, -7])
>>> s
0    1
1    2
2   -7
dtype: int64
Estos son enteros pequeños, entonces, ¿qué hay de convertir a un tipo de 8 bits sin firmar para ahorrar memoria?

>>> s.astype(np.uint8)
0      1
1      2
2    249
dtype: uint8
La conversión funcionó, pero el -7 se redondeó para convertirse en 249 (es decir, 28 - 7)!Manejo de errores
Pero, ¿qué pasa si algunos valores no se pueden convertir a un tipo numérico?

to_numeric() también toma un argumento de palabra clave errors que le permite forzar que los valores no numéricos sean NaN, o simplemente ignorar las columnas que contienen estos valores.

Aquí hay un ejemplo que usa una serie de cadenas s que tiene el tipo de objeto:

>>> s = pd.Series(['1', '2', '4.7', 'pandas', '10'])
>>> s
0         1
1         2
2       4.7
3    pandas
4        10
dtype: object
El comportamiento predeterminado es aumentar si no puede convertir un valor. En este caso, no puede hacer frente a la cadena 'pandas':

>>> pd.to_numeric(s) # or pd.to_numeric(s, errors='raise')
ValueError: Unable to parse string
En lugar de fallar, podríamos querer que 'pandas' se considere un valor numérico faltante/malo. Podemos forzar valores no válidos a NaN de la siguiente manera usando el argumento de la palabra clave errors:

>>> pd.to_numeric(s, errors='coerce')
0     1.0
1     2.0
2     4.7
3     NaN
4    10.0
dtype: float64
La tercera opción para errors es ignorar la operación si se encuentra un valor no válido:

>>> pd.to_numeric(s, errors='ignore')
# the original Series is returned untouched
Esta última opción es particularmente útil cuando desea convertir todo su DataFrame, pero no sabe cuál de nuestras columnas se puede convertir de manera confiable a un tipo numérico. En ese caso solo escribe:

df.apply(pd.to_numeric, errors='ignore')
La función se aplicará a cada columna del DataFrame. Las columnas que se pueden convertir a un tipo numérico se convertirán, mientras que las columnas que no pueden (por ejemplo, que contienen cadenas o fechas sin dígitos) se dejarán solas.

Downcasting
De forma predeterminada, la conversión con to_numeric() le dará un dtype int64 o float64 (o cualquier ancho de entero que sea nativo de su plataforma).

Eso suele ser lo que quiere, pero ¿qué sucede si desea guardar algo de memoria y usar un tipo de letra más compacto, como float32, o int8?

to_numeric() le da la opción de cambiar a 'entero', 'firmado', 'sin firmar', 'flotar'. Aquí hay un ejemplo para una serie simple s de tipo entero:

>>> s = pd.Series([1, 2, -7])
>>> s
0    1
1    2
2   -7
dtype: int64
La reducción a "entero" utiliza el entero más pequeño posible que puede contener los valores:

>>> pd.to_numeric(s, downcast='integer')
0    1
1    2
2   -7
dtype: int8
La reducción a 'flotar' elige de manera similar un tipo flotante más pequeño que el normal:

>>> pd.to_numeric(s, downcast='float')
0    1.0
1    2.0
2   -7.0
dtype: float32
2. astype()
El método astype() le permite ser explícito sobre el tipo de dato que desea que tenga su DataFrame o Series. Es muy versátil, ya que puede intentar ir de un tipo a otro.

Uso básico
Simplemente elija un tipo: puede usar un tipo de tipo NumPy (por ejemplo, np.int16), algunos tipos de Python (por ejemplo, bool) o tipos específicos de pandas (como el tipo de categórico).

Llame al método en el objeto que desea convertir y astype() intentará convertirlo por usted:

# convert all DataFrame columns to the int64 dtype
df = df.astype(int)

# convert column "a" to int64 dtype and "b" to complex type
df = df.astype({"a": int, "b": complex})

# convert Series to float16 type
s = s.astype(np.float16)

# convert Series to Python strings
s = s.astype(str)

# convert Series to categorical type - see docs for more details
s = s.astype('category')
Observe que dije "intentar": si astype() no sabe cómo convertir un valor en la serie o el marco de datos, generará un error. Por ejemplo, si tiene un valor NaN o inf, obtendrá un error al intentar convertirlo en un entero.

A partir de las pandas 0.20.0, este error se puede eliminar al pasar errors='ignore'. Su objeto original será devuelto sin tocar.

Ten cuidado
astype() es potente, pero a veces convertirá los valores "incorrectamente". Por ejemplo:

>>> s = pd.Series([1, 2, -7])
>>> s
0    1
1    2
2   -7
dtype: int64
Estos son enteros pequeños, entonces, ¿qué hay de convertir a un tipo de 8 bits sin firmar para ahorrar memoria?

>>> s.astype(np.uint8)
0      1
1      2
2    249
dtype: uint8
La conversión funcionó, pero el -7 se redondeó para convertirse en 249 (es decir, 28 - 7)!

Intentar hacer un downcast usando pd.to_numeric(s, downcast='unsigned') podría ayudar a prevenir este error.

3. infer_objects()
La versión 0.21.0 de pandas introdujo el método infer_objects() para convertir columnas de un DataFrame que tienen un tipo de datos de objeto a un tipo más específico (conversiones de software).

Por ejemplo, aquí hay un DataFrame con dos columnas de tipo de objeto. Uno contiene números enteros reales y el otro contiene cadenas que representan números enteros:

>>> df = pd.DataFrame({'a': [7, 1, 5], 'b': ['3','2','1']}, dtype='object')
>>> df.dtypes
a    object
b    object
dtype: object
Usando infer_objects(), puede cambiar el tipo de columna 'a' a int64:

>>> df = df.infer_objects()
>>> df.dtypes
a     int64
b    object
dtype: object
La columna 'b' se ha dejado sola ya que sus valores eran cadenas, no enteros. Si quisiera probar y forzar la conversión de ambas columnas a un tipo entero, podría usar df.astype(int) en su lugar.

Intentar hacer un downcast usando pd.to_numeric(s, downcast='unsigned') podría ayudar a prevenir este error.

3. infer_objects()
La versión 0.21.0 de pandas introdujo el método infer_objects() para convertir columnas de un DataFrame que tienen un tipo de datos de objeto a un tipo más específico (conversiones de software).

Por ejemplo, aquí hay un DataFrame con dos columnas de tipo de objeto. Uno contiene números enteros reales y el otro contiene cadenas que representan números enteros:

>>> df = pd.DataFrame({'a': [7, 1, 5], 'b': ['3','2','1']}, dtype='object')
>>> df.dtypes
a    object
b    object
dtype: object
Usando infer_objects(), puede cambiar el tipo de columna 'a' a int64:

>>> df = df.infer_objects()
>>> df.dtypes
a     int64
b    object
dtype: object
La columna 'b' se ha dejado sola ya que sus valores eran cadenas, no enteros. Si quisiera probar y forzar la conversión de ambas columnas a un tipo entero, podría usar df.astype(int) en su lugar.

In [ ]:
a = [['a', '1.2', '4.2'], ['b', '70', '0.03'], ['x', '5', '0']]
df = pd.DataFrame(a, columns=['one', 'two', 'three'])
df
Out[16]: 
  one  two three
0   a  1.2   4.2
1   b   70  0.03
2   x    5     0

df.dtypes
Out[17]: 
one      object
two      object
three    object

df[['two', 'three']] = df[['two', 'three']].astype(float)

df.dtypes
Out[19]: 
one       object
two      float64
three    float64